# Tripadvisor Hotel

## Prerequisites

In [ ]:
#%pip install helium
#%pip install bs4
#%pip install pandas
#%pip install requests

## Web scraping

### Import

In [ ]:
from multiprocessing import Pool
from time import sleep
from helium import *
from bs4 import BeautifulSoup
import pandas as pd
import multiprocessing as mp
import requests
import shutil
import os
import csv

### Constants

In [ ]:
COLUMNS_NAME = ['HOTEL_NAME', 'HOTEL_RATING', 'HOTEL_PRICE', 'HOTEL_LOCATION', 'HOTEL_REVIEW_LINK', 'REVIEW_RATING', 'REVIEW_DATE', 'REVIEW_TEXT']
ROOT_URL = 'https://www.tripadvisor.com/'
MAX_HOTEL_PAGES = 1
MAX_REVIEW_PAGES = 200

In [ ]:
try:
    writer = csv.DictWriter(open('tripadvisor_hotels_data.csv', 'w', encoding='UTF8'), fieldnames=COLUMNS_NAME)
    writer.writeheader()
except IOError:
    print("I/O error")

### Methods

In [ ]:
def get_reviews_info(reviews, row, page):
    # Expand the review
    scroll_down(4500)
    wait_until(S("//span[text()='Read more']").exists)
    sleep(7)
    
    try:
        click('Read more')
        for x in range(len(reviews)):
            review = reviews[x]
            # REVIEW_RATING
            row[COLUMNS_NAME[5]] = review.find('div', class_='Hlmiy F1').find_all('span')[0].get('class')[1]
            # REVIEW_DATE
            row[COLUMNS_NAME[6]] = review.find('span', class_='teHYY _R Me S4 H3').text
            # REVIEW_TEXT
            row[COLUMNS_NAME[7]] = review.find('div', class_='fIrGe _T').text
            try:
                print('Hotel: ' + row[COLUMNS_NAME[0]] + ' - ' + 'Review: ' + str(x+1) + ' - Page: ' + str(page))
                writer.writerow(row)
            except IOError:
                print("I/O error")
    except:
        print('ERROR: ' + 'Hotel: ' + row[COLUMNS_NAME[0]] + ' - ' + 'Review: ' + str(x+1) + ' - Page: ' + str(page+1))


In [ ]:
def get_hotels_info(hotels, max_review_pages):
    for x in range(len(hotels)):
        hotel = hotels[x]
        row = {}

        # HOTEL_NAME
        row[COLUMNS_NAME[0]] = hotel.find('a', class_='property_title prominent').text
        # HOTEL_RATING
        row[COLUMNS_NAME[1]] = hotel.find('div', class_='prw_rup prw_common_rating_and_review_count_with_popup linespace is-shown-at-mobile').find_all('a')[0].get('alt')
        # HOTEL_PRICE
        row[COLUMNS_NAME[2]] = hotel.find('div', class_='price __resizeWatch').text
        # HOTEL_REVIEW_LINK
        row[COLUMNS_NAME[4]] = ROOT_URL + hotel.find('a', class_='review_count').get('href')
        
        # Goes to the hotel's reviews
        browser = start_firefox(url=row[COLUMNS_NAME[4]], headless=True)
        wait_until(S("//div[@class='jvqAy']").exists)
        sleep(10)
        reviews_soup = BeautifulSoup(browser.page_source, 'html.parser')

        # HOTEL_LOCATION
        row[COLUMNS_NAME[3]] = reviews_soup.find('span', class_='fHvkI PTrfg').text
        
        for y in range(max_review_pages):
            reviews_soup = BeautifulSoup(browser.page_source, 'html.parser')
            get_reviews_info(reviews_soup.find_all('div', class_='YibKl MC R2 Gi z Z BB pBbQr'), row, y)
            # Goes to the next reviews page
            go_to(ROOT_URL + reviews_soup.find('a', class_='ui_button nav next primary').get('href'))
            
        kill_browser()
        

In [ ]:
def get_data(threadNumber, URL, max_hotel_pages):
    errors = 0
    browser = start_firefox(url=URL, headless=True)
    # Press "See all" button to show the navigation menu
    wait_until(S("//span[text()='Check In']").exists)
    sleep(3)
    press(END)
    scroll_up(600)
    wait_until(Button('See all').exists)
    sleep(2)
    click('See all')
    sleep(15)

    for x in range(max_hotel_pages):
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        get_hotels_info(soup.find_all('div', class_='ui_column is-8 main_col allowEllipsis'), MAX_REVIEW_PAGES)
        set_driver(browser)
        press(END)
        click('Next')
        wait_until(S("//span[text()='Check In']").exists)
        sleep(3)
        
    kill_browser()

### Getting the data

In [ ]:
get_data(4, 'https://www.tripadvisor.com/Hotels-g4-Europe-Hotels.html', MAX_HOTEL_PAGES)